In [ ]:
import pandas as pd
import nltk

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Загрузка словаря
И его подготовка


In [ ]:
dic = pd.read_csv('drive/MyDrive/ОБЩЕЕ/РСОДПО/AI/emotional/data/kartaslovsent.csv', sep=';')
dic

,term,tag,value,pstv,ngtv,neut,dunno,pstvNgtvDisagreementRatio
0,абажур,NEUT,0.08,0.185,0.037,0.580,0.198,0.00
1,аббатство,NEUT,0.10,0.192,0.038,0.578,0.192,0.00
2,аббревиатура,NEUT,0.08,0.196,0.000,0.630,0.174,0.00
3,абзац,NEUT,0.00,0.137,0.000,0.706,0.157,0.00
4,абиссинец,NEUT,0.28,0.151,0.113,0.245,0.491,0.19
...,...,...,...,...,...,...,...,...
46122,ёмкость,NEUT,0.00,0.167,0.000,0.690,0.143,0.00
46123,ёрзать,NGTV,-0.54,0.050,0.446,0.397,0.107,0.00
46124,ёрничать,NGTV,-0.79,0.078,0.529,0.236,0.157,0.00
46125,ёрш,NEUT,0.16,0.224,0.072,0.576,0.128,0.00


In [ ]:
dic = dic[['term', 'tag', 'value', 'pstvNgtvDisagreementRatio']]
dic

,term,tag,value,pstvNgtvDisagreementRatio
0,абажур,NEUT,0.08,0.00
1,аббатство,NEUT,0.10,0.00
2,аббревиатура,NEUT,0.08,0.00
3,абзац,NEUT,0.00,0.00
4,абиссинец,NEUT,0.28,0.19
...,...,...,...,...
46122,ёмкость,NEUT,0.00,0.00
46123,ёрзать,NGTV,-0.54,0.00
46124,ёрничать,NGTV,-0.79,0.00
46125,ёрш,NEUT,0.16,0.00


In [ ]:
positives = dic[(dic == 'PSTV').any(axis=1)][['term', 'value']]
neutrals  = dic[(dic == 'NEUT').any(axis=1)][['term', 'value']]
negatives = dic[(dic == 'NGTV').any(axis=1)][['term', 'value']]

# Функции "определения" тональности

In [ ]:
def determine1(row):
  pstv = 0
  neut = 0
  ngtv = 0
  if (row['lemm'] is not None):
    for word in row['lemm']:
      if word in positives.values:
        pstv += 1
      elif word in neutrals.values:
        neut += 1
      elif word in negatives.values:
        ngtv += 1
    # print([ngtv, neut, pstv])
    return [ngtv, neut, pstv].index(max([ngtv, neut, pstv]))
  else:
    return None

In [ ]:
import numpy as np

def determine2(row, is_num=False):
  ngtv = 0
  neut = 0
  pstv = 0
  em_decode = ['NGTV', 'NEUT', 'PSTV']
  for index, word in dic[np.isin(dic, [row['lemm']]).any(axis=1)].iterrows():
    if word['tag'] == 'NGTV':
      ngtv += word['value']
    elif word['tag'] == 'NUET':
      neut += word['value']
    elif word['tag'] == 'PSTV':
      pstv += word['value']
  if (is_num):
    return [ngtv, neut, pstv].index(max([ngtv, neut, pstv]))
  else:
    return em_decode[[ngtv, neut, pstv].index(max([ngtv, neut, pstv]))]

# Работа с текстом

In [ ]:
pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 1.8 MB/s 
     |████████████████████████████████| 8.2 MB 6.7 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=e87bfce1c1d10ed2f4439925f3a4a6a709e36098d095615d38e3b37c7a42b8ed
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built docopt


In [ ]:
import re
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords
nltk.download('stopwords')

patterns = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
stopwords_ru = stopwords.words("russian")
morph = MorphAnalyzer()

def lemmatize(doc):
    doc = re.sub(patterns, ' ', doc)
    tokens = []
    for token in doc.split():
        if token and token not in stopwords_ru:
            token = token.strip()
            token = morph.normal_forms(token)[0]
            
            tokens.append(token)
    if len(tokens) > 2:
        return tokens
    return None

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# df = pd.read_csv('drive/MyDrive/ОБЩЕЕ/РСОДПО/AI/key_tokens/data/feedbacks_dropna_sorted_cleaned.csv', sep=';')[['На какой электив вы хотите оставить отзыв?', 'Введите текст отзыва на электив', 'На сколько бы вы оценили этот электив?', 'Ваши ожидания от этого электива оправдались?', 'На сколько бы вы оценили этого преподавателя в рамках этого электива?']]
df = pd.read_csv('drive/MyDrive/ОБЩЕЕ/РСОДПО/AI/key_tokens/data/feedbacks_dropna_sorted_cleaned.csv', sep=';')[['На какой электив вы хотите оставить отзыв?', 'Введите текст отзыва на электив']]
corpus = df['Введите текст отзыва на электив']

In [ ]:
%%time
df['lemm'] = corpus.apply(lambda x: lemmatize(x))
df['emote'] = df.apply(determine1, axis = 1)
# ngtv, neut, pstv

CPU times: user 40.7 s, sys: 113 ms, total: 40.8 s
Wall time: 53.3 s


In [ ]:
df.drop('lemm', axis=1)

,На какой электив вы хотите оставить отзыв?,Введите текст отзыва на электив,emote
0,3D и медийная реклама в программе 3DS MAX,"Чисто основы основ 3д моделинга, для тех кто в...",1.0
1,Python - программная среда для аналитиков,Кайфы ловлю,NaN
2,R для лингвистов: программирование и анализ да...,Проходил он онлайн на платформе openedu.ru. По...,1.0
3,Адаптация населения России к трудностям револю...,Супер пушка класс препод супер интересные исто...,2.0
4,Анализ данных в R,"Анализируем данные на ЯП R, на автомат выйти с...",1.0
...,...,...,...
192,Экология. Технологии. Аниме/Ecology. Technolog...,"Плюс в том, что можешь пройти когда хочешь, ми...",1.0
193,Эристика,"Помимо коммуникативных навыков, даёт неплохой ...",2.0
194,Эристика,"Электив, на котором учат искусству спора и пуб...",1.0
195,Эристика,Классный электив для любителей поработать язык...,1.0


In [ ]:
%%time
df['lemm'] = corpus.apply(lambda x: lemmatize(x))
df['emote'] = df.apply(determine2, axis = 1)
# ngtv, neut, pstv

CPU times: user 35.1 s, sys: 116 ms, total: 35.2 s
Wall time: 37.6 s


In [ ]:
df.drop('lemm', axis=1)

,На какой электив вы хотите оставить отзыв?,Введите текст отзыва на электив,emote
0,3D и медийная реклама в программе 3DS MAX,"Чисто основы основ 3д моделинга, для тех кто в...",PSTV
1,Python - программная среда для аналитиков,Кайфы ловлю,NGTV
2,R для лингвистов: программирование и анализ да...,Проходил он онлайн на платформе openedu.ru. По...,PSTV
3,Адаптация населения России к трудностям револю...,Супер пушка класс препод супер интересные исто...,PSTV
4,Анализ данных в R,"Анализируем данные на ЯП R, на автомат выйти с...",PSTV
...,...,...,...
192,Экология. Технологии. Аниме/Ecology. Technolog...,"Плюс в том, что можешь пройти когда хочешь, ми...",NGTV
193,Эристика,"Помимо коммуникативных навыков, даёт неплохой ...",PSTV
194,Эристика,"Электив, на котором учат искусству спора и пуб...",PSTV
195,Эристика,Классный электив для любителей поработать язык...,PSTV


# F1-мера

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
test = pd.read_csv('drive/MyDrive/ОБЩЕЕ/РСОДПО/AI/emotional/data/train.csv', sep=',', nrows=10000, on_bad_lines='skip')
test

,Unnamed: 0,text,sentiment
0,21098,".с.,и спросил его: о Посланник Аллаха!Ты пори...",1
1,21099,Роднее всех родных Попала я в ГКБ №8 еще в дек...,1
2,21100,Непорядочное отношение к своим работникам Рабо...,2
3,21101,"). Отсутствуют нормативы, Госты и прочее, что ...",1
4,21102,У меня машина в руках 5 лет и это п...,1
...,...,...,...
9995,31093,Огромное спасибо врачам и медсёстрам 6-го акуш...,1
9996,31094,"Заведение приличное , только пить пиво нельзя ...",0
9997,31095,"Сейчас мы тут, номер 7108 с видом на дорогу) з...",1
9998,31096,"Врач Коваленченко, грубое отношение к пациента...",2


In [ ]:
test = test.replace({'sentiment' : { 2 : 0, 0 : 1, 1 : 2}})
# NEUT, PSTV, NGTV -> NGTV, NEUT, PSTV

In [ ]:
corpus = test['text']

In [ ]:
%%time
test['lemm'] = corpus.apply(lambda x: lemmatize(x))
test['emote'] = test.apply(lambda x: determine2(x, is_num=True), axis = 1)
# ngtv, neut, pstv

CPU times: user 1h 14min 42s, sys: 16.3 s, total: 1h 14min 58s
Wall time: 1h 15min 6s


In [ ]:
test.drop('lemm', axis=1)

,Unnamed: 0,text,sentiment,emote
0,21098,".с.,и спросил его: о Посланник Аллаха!Ты пори...",2,2
1,21099,Роднее всех родных Попала я в ГКБ №8 еще в дек...,2,2
2,21100,Непорядочное отношение к своим работникам Рабо...,0,2
3,21101,"). Отсутствуют нормативы, Госты и прочее, что ...",2,2
4,21102,У меня машина в руках 5 лет и это п...,2,2
...,...,...,...,...
9995,31093,Огромное спасибо врачам и медсёстрам 6-го акуш...,2,2
9996,31094,"Заведение приличное , только пить пиво нельзя ...",1,2
9997,31095,"Сейчас мы тут, номер 7108 с видом на дорогу) з...",2,2
9998,31096,"Врач Коваленченко, грубое отношение к пациента...",0,2


In [ ]:
test['sentiment'].value_counts()

2    4856
1    2610
0    2534
Name: sentiment, dtype: int64

In [ ]:
y_true = test['sentiment']
y_pred = test['emote']

In [ ]:
f1_score(y_true, y_pred, average='macro')

0.2524931006246871

In [ ]:
f1_score(y_true, y_pred, average='micro')

0.4955

In [ ]:
f1_score(y_true, y_pred, average='weighted')

0.34683256696942927

На 10000 записей F1-мера составила:
*    Macro | 0.2524931006246871
*    Micro | 0.4955
* Weighted | 0.34683256696942927

Временные затраты 1 час 15 минут 6 секунд 

[CPU times: user 1h 14min 42s, sys: 16.3 s, total: 1h 14min 58s Wall time: 1h 15min 6s]

ПЛОХО и немного БОЛЬНО

# "Эмпирическая" мера

In [ ]:
from sklearn.metrics import precision_recall_fscore_support as alotmetrics

In [ ]:
of = pd.read_csv('https://docs.google.com/spreadsheets/d/1kD4JRC_nDcF2ysmXKgYTA6jNpUi3vdbSmJY99lJxZis/export?format=csv')[['text', 'y']].replace({'y' : { -1 : 'NGTV', 0 : 'NEUT', 1 : 'PSTV'}})
of.columns = ['text', 'true']
of

,text,true
0,Электив сам по себе скучноватый.,NGTV
1,"Я не считаю, что полученные знания помогут мне...",NGTV
2,Преподаватель очень тихо ведёт лекцию.,NEUT
3,Преподаватель не пытается как-то ухудшить проц...,NEUT
4,"Ничего не делаем, в конце 2 проекта, можно бра...",NEUT
...,...,...
138,Такая теория никак не поможет ребятам заинтере...,NGTV
139,"Было бы гораздо лучше, будь преподаватель с ка...",NGTV
140,Или это преподавали бы люди с кафедры предприн...,NEUT
141,"Толку учить пользоваться программой, где самый...",NGTV


In [ ]:
%%time
of['lemm'] = corpus.apply(lambda x: lemmatize(x))
of['pred'] = of.apply(determine2, axis = 1)
# ngtv, neut, pstv

CPU times: user 24.7 s, sys: 92.8 ms, total: 24.8 s
Wall time: 25.9 s


In [ ]:
of

,text,true,lemm,pred
0,Электив сам по себе скучноватый.,NGTV,"[чисто, основа, основа, далее, моделинга, тот,...",PSTV
1,"Я не считаю, что полученные знания помогут мне...",NGTV,None,NGTV
2,Преподаватель очень тихо ведёт лекцию.,NEUT,"[проходить, онлайн, платформа, попасть, курс, ...",PSTV
3,Преподаватель не пытается как-то ухудшить проц...,NEUT,"[супер, пушка, класс, препод, супер, интересны...",PSTV
4,"Ничего не делаем, в конце 2 проекта, можно бра...",NEUT,"[анализировать, дать, яп, автомат, выйти, слож...",PSTV
...,...,...,...,...
138,Такая теория никак не поможет ребятам заинтере...,NGTV,"[закрыться, легко, просто, посещать, занятие, ...",PSTV
139,"Было бы гораздо лучше, будь преподаватель с ка...",NGTV,"[много, гость, предприниматель, иногда, дельны...",PSTV
140,Или это преподавали бы люди с кафедры предприн...,NEUT,"[лекция, нудный, скучный, преподаватель, некот...",PSTV
141,"Толку учить пользоваться программой, где самый...",NGTV,"[прекрасный, электив, помогать, разобраться, д...",PSTV


In [ ]:
print('Precision', 'Recall', 'F-score', 'HZ')
print(alotmetrics(of['true'], of['pred'], average='macro'))
print(alotmetrics(of['true'], of['pred'], average='micro'))
print(alotmetrics(of['true'], of['pred'], average='weighted'))

Precision Recall F-score HZ
(0.4529262086513995, 0.36384845463609167, 0.2577684235142798, None)
(0.3706293706293706, 0.3706293706293706, 0.3706293706293706, None)
(0.4506218971867827, 0.3706293706293706, 0.25307044091574477, None)
